In [1]:
# Install muscle3.8 for biopython
# wget https://drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz
# tar xvf muscle3.8.31_i86linux64.tar.gz 
# sudo cp muscle3.8.31_i86linux64 /usr/bin/muscle
# sudo chmod +x /usr/bin/muscle

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import MuscleCommandline
from pathlib import Path
import pandas as pd
from collections import namedtuple

In [3]:
REF_SEQ = "PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDEDFRKYTAFTIPSINNETPGIRYQYNVLPQGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRTKIEELRQHLLRWGLTTPDKKHQKEPPFLWMGYEL"
REF_SEQ_LEN = len(REF_SEQ)
data_path = Path("/data/hiv/data")

In [4]:
Mutation = namedtuple("Mutation", "original index mutation")

# At least 4 mutations among
mutations1 = [
    Mutation("M", 40, ["L"]),
    Mutation("E", 41, ["D"]),
    Mutation("D", 66, ["N"]),
    Mutation("T", 68, ["D", "N", "S"]),
    Mutation("L", 73, ["V", "I"]),
    Mutation("L", 209, ["W"]),
    Mutation("T", 214, ["A", "C", "D", "E", "G", "H", "I", "L", "N", "S", "V", "Y", "F"])
]

mutations2 = [
    Mutation("K", 64, ["R", "E", "N"])
]

mutations3 = [
# insertion at codon 69    
]

mutations4 = [
    Mutation("K", 69, ["E"])    
]

In [5]:
df = pd.read_csv("/data/hiv/data/pol/2-pol-20000-aligned.csv")
df

,accession,gene
0,FJ199594,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKAL--TAICEEMEKE...
1,AB874173,PISPIETVPVKLKPGMDGPKVRQWPLTEEKIKAL--VEICTEMEKE...
2,GQ272384,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKAL--TEICAEMEKE...
3,FJ199763,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKAL--TAICEEMEKE...
4,FJ199639,PISSIETVPVKLKPGMDGPKVKQWPLTEEKIKAL--TAICEEMEKE...
...,...,...
19994,GQ288367,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALV--EICTEMEKE...
19995,GQ288349,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALV--EICTEMEKE...
19996,GQ288353,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALV--EICTEMEKE...
19997,GQ290758,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALV--EICTEMEKE...


In [8]:
def align_sequences(reference_seq, target_seq, working_dir="tmp"):
           
    # Muscle uses files, need directory for temporary files
    working_dir = Path(working_dir)
    working_dir.mkdir(parents=True, exist_ok=True)
    
    # Input file contains two sequenes: first reference, second target sequence to compare with
    input_file = working_dir.joinpath("test.fasta")
    aligned_file = working_dir.joinpath("test_aligned.fasta")

    sequences = [SeqRecord(Seq(reference_seq), id="reference"), SeqRecord(Seq(target_seq), id="target")]
    SeqIO.write(sequences, input_file, "fasta")
    
    # Alignment
    muscle_cline = MuscleCommandline(input=input_file, out=aligned_file)
    muscle_cline()

    reference_aln = None
    target_aln = None
    
    # Read aligned sequences
    aligned = SeqIO.parse(aligned_file, "fasta")
    for seq in aligned:
        if seq.id == "reference":
            reference_aln = seq
        elif seq.id == "target":
            target_aln = seq

    # Reference sequence map of {original:aligned} indices
    index_map = {}
    index_o = 0
    for index_aln, c in enumerate(reference_aln.seq):
        if c == "-":
            continue
        index_map[index_o] = index_aln
        index_o += 1
        
    return reference_aln, target_aln, index_map

def find_mutations(mutations, reference_aln, target_aln, index_map, verbose=False):
    
    n_found = 0
    
    for mutation in mutations:
        reference_letter = reference_aln[index_map[mutation.index]]
        target_letter = target_aln[index_map[mutation.index]]
        
        # Make sure that reference sequence is correct
        if reference_letter != mutation.original:
            if verbose:
                print(f"Mutation ({mutation}) original letter does not match")
            continue
        
        # Check if target letter at mutation index is one of expected mutations
        if target_letter in mutation.mutation:
            if verbose:
                print(f"Index: ({mutation.index}) : Found mutation")
            n_found += 1
        elif target_letter != reference_letter:
            if verbose:
                print(f"Index: ({mutation.index}) : Different from reference, but not a mutation. Reference: [{reference_letter}] Target: [{target_letter}]")

    return n_found


for i in range(1000):
    
    
    reference_aln, target_aln, index_map = align_sequences(REF_SEQ, df.iloc[i].gene)
    m1 = find_mutations(mutations1, reference_aln, target_aln, index_map)
    m2 = find_mutations(mutations2, reference_aln, target_aln, index_map)
    m4 = find_mutations(mutations4, reference_aln, target_aln, index_map)
#     print(f"{i}) m1:{m1}, m2:{m2}, m4:{m4}")
    
    if m1 >= 4 or m2 == 1 or m4 == 1:
        print(f"{i}) {df.iloc[i].accession} has resistance. m1:{m1}, m2:{m2}, m4:{m4}")
    elif m1 >= 3:
        print(f"{i}) {df.iloc[i].accession} possible resistance. m1:{m1}, m2:{m2}, m4:{m4}")
        
    
print("Done")

48) AB253423 has resistance
56) AB253682 has resistance
57) AB253684 has resistance
58) AB253686 has resistance
59) AB253687 has resistance
60) AB253688 has resistance
61) AB253690 has resistance
62) AB253695 has resistance
63) AB253703 has resistance
64) AB253704 has resistance
65) AB253710 has resistance
66) AB253714 has resistance
67) AB253717 has resistance
68) AB253719 has resistance
74) AB254148 has resistance
173) AB287369 has resistance
174) AB287371 has resistance
175) AB287372 has resistance
178) AB289587 possible resistance
185) AB356103 has resistance
197) AB356119 possible resistance
585) AB442317 possible resistance
625) AB480298 has resistance
626) AB480300 has resistance
627) AB480301 has resistance
628) AB480692 has resistance
629) AB480693 has resistance
630) AB480694 has resistance
631) AB480695 has resistance
634) AB480698 possible resistance
667) AB564744 has resistance
669) AB564746 has resistance
689) AB604946 has resistance
